In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import librosa
import glob
import IPython.display as ipd
from csmssmtools import *
from cover_alignment import *
from seqalignment import *
from audioutils import *
import crema
import IPython.display as ipd

In [ ]:
import glob
pairs = glob.glob("covers80_audio/*")
pairs = [p for p in pairs if not "list" in p]
tunes  = [glob.glob("{}/*".format(p))[0] for p in pairs]
tunes += [glob.glob("{}/*".format(p))[1] for p in pairs]

In [ ]:
cremas = []
model = crema.models.chord.ChordModel()
for tune in tunes:
    y, sr = librosa.load(tune)
    cremas.append(model.outputs(y=y, sr=sr)['chord_pitch'].T)

In [ ]:
avg_fac = 10
cremas = [aggregate_intervals(c, avg_fac, [np.median]) for c in cremas]

### Crema Similarity Covers80

All pairs alignments

In [ ]:
kappa = 0.095
wins_per_block = 20
sim_crema = np.zeros((80, 80))
for i in range(80):
    for j in range(80):
        ipd.clear_output()
        print(i, j)
        sim_crema[i, j] = get_alignment_score_serra(chrompwr(cremas[i], 0.5), chrompwr(cremas[j+80], 0.5), wins_per_block)

In [ ]:
plt.imshow(sim_crema)
correct = np.argmax(sim_crema, axis=1) == np.arange(sim_crema.shape[0])
print(np.sum(correct)) # 30 on first try, 45 with OTI